<img src="./images/DLI_Header.png" style="width: 400px;">

# 美國手語資料集的影像分類

在本節中，我們會執行在上一節看到的資料準備、模型建立和模型訓練步驟，但使用不同的資料集：用手勢比出[美國手語](http://www.asl.gs/)字母的影像。

## 目標

* 準備用於訓練的影像資料
* 建立並編寫簡單的影像分類模型
* 訓練影像分類模型並觀察結果

## 美國手語資料集

[美國手語字母表](http://www.asl.gs/)含有 26 個字母。其中兩個字母 (j 和 z) 需要移動，因此不會包含在訓練資料集中。

<img src="./images/asl.png" style="width: 600px;">

### Kaggle

此資料集可從網站 [Kaggle](http://www.kaggle.com) 取得，這裡是尋找資料集和其他深度學習資源的絕佳空間。除了提供資料集和「核心」(類似此課程的Notebooks)等資源外，Kaggle 也會舉辦可參加的比賽，讓參加者在訓練高度精確的模型上彼此競爭。

如果你想要練習或查看眾多深度學習專案的範例，Kaggle 是值得造訪的絕佳網站。

## 載入資料

這個資料集透過 Keras 的取得方式和 MNIST 不同，所以讓我們瞭解一下如何載入自訂資料。完成本節之後，我們會和先前一樣有 `x_train`、`y_train`、`x_valid` 和 `y_valid` 變數。

### 讀取資料

手語資料集是 [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) (逗號分隔) 格式，Microsoft Excel 和 Google 試算表都是採用這種資料結構。這是由欄和列組成的網格，頂部會有欄位名稱，就像在 [訓練](asl_data/sign_mnist_train.csv) 和[驗證](asl_data/sign_mnist_valid.csv)資料集看到的一樣 (可能需要花一些時間載入)。

若要載入和使用資料，我們必須使用名為 [Pandas](https://pandas.pydata.org/) 的函式庫，這是可載入和操作資料的高效能工具。我們會將 CSV 檔案讀入一種稱為 [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) 的格式。

In [ ]:
import pandas as pd

Pandas 的 [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) 方法可接收 CSV 檔案，並回傳 DataFrame 檔案：

In [ ]:
train_df = pd.read_csv("asl_data/sign_mnist_train.csv")
valid_df = pd.read_csv("asl_data/sign_mnist_valid.csv")

### 探索資料

讓我們來看看目前的資料。我們可以使用 [head](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) 方法來列印 DataFrame 的前幾列。每列都是一張影像，並包含一個 `label`欄，此外還有 784 個值代表影像中的每個像素值，和 MNIST 資料集一樣。請注意，目前的影像類別都是數值，而非字母表中的字母：

In [ ]:
train_df.head()

### 擷取影像類別

和 MNIST 一樣，我們想要將訓練與驗證影像類別儲存為 `y_train`和 `y_valid` 變數。現在我們要建立這些變數，然後從原始 DataFrame 檔案刪除類別資料，因為檔案不再需要這些類別資料：

In [ ]:
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

### 擷取影像

和 MNIST 一樣，我們想將訓練與驗證影像儲存為 `x_train`和 `x_valid` 變數。現在我們要建立這些變數：

In [ ]:
x_train = train_df.values
x_valid = valid_df.values

### 總結訓練與驗證資料

我們現在有 27,455 張影像，各有 784 像素，可以用於訓練...

In [ ]:
x_train.shape

...以及其對應的影像類別：

In [ ]:
y_train.shape

在驗證方面，我們有 7,172 張影像...

In [ ]:
x_valid.shape

...及其對應的影像類別：

In [ ]:
y_valid.shape

## 視覺化資料

為了視覺化影像，我們要再次使用 matplotlib 函式庫。我們不需要顧慮視覺化的細節，但如果你有興趣，可以在稍後深入瞭解 [matplotlib](https://matplotlib.org/)。

請注意，我們必須將資料從目前的 784 像素 1D 形狀，重新調整成 28x28 像素的 2D 形狀，讓影像變得有意義：

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(40,40))

num_images = 20
for i in range(num_images):
    row = x_train[i]
    label = y_train[i]
    
    image = row.reshape(28,28)
    plt.subplot(1, num_images, i+1)
    plt.title(label, fontdict={'fontsize': 30})
    plt.axis('off')
    plt.imshow(image, cmap='gray')

## 練習：正規化影像資料

與 MNIST 資料集一樣，我們會將影像資料正規化，這表示影像的像素值，並不會介於目前的 0 到 255：

In [ ]:
x_train.min()

In [ ]:
x_train.max()

...而應該要是介於 0 和 1 的浮點值。使用下列儲存格執行作業。如果你遇到困難，請參閱下方的解決方案。

In [ ]:
# TODO: Normalize x_train and x_valid.

### 解決方案

按一下以下的「...」以顯示解決方案。

```python
x_train = x_train / 255
x_valid = x_valid / 255
```

## 練習：影像分類

就像我們先前對 MNIST 資料集所採取的動作，現在要對影像類別進行分類編碼。回想一下，我們可以使用 [keras.utils.to_categorical](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) 方法來轉換編碼的值以及類別數量，以完成編碼作業。請在下方的儲存格中完成作業。我們已經幫你匯入 `keras` 並設定好類別數量 (24)。

In [ ]:
import tensorflow.keras as keras
num_classes = 24

In [ ]:
# TODO: Categorically encode y_train and y_valid.

### 解決方案

按一下以下的「...」以顯示解決方案。

```python
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)
```

## 練習：建立模型

資料全都準備好了，我們已將用於訓練和驗證的影像正規化，也將用於訓練和驗證的影像類別進行分類編碼。

在本練習中，我們要建立循序模型。就像上次一樣，我們所建立的模型：
* 具有密集輸入層。這一層包含 512 個神經元，使用 `relu`激活函數，並接收形狀為 `(784,)` 的輸入影像。
* 具有第二個密集輸入層，含有 512 個使用 `relu`激活函數的神經元
* 具有密集輸出層，其神經元的數量與類別數量相等，並使用 `softmax`激活函數

請在下方的儲存格中執行作業，建立 `model`變數以儲存模型。我們已經匯入 Keras [循序](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential)模型類別和[密集](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense)層類別，因此你可以立即開始使用。如需提示，請展開下方的解決方案：

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# TODO: build a model following the guidelines above.

### 解決方案

按一下以下的「...」以顯示解決方案。

```python
model = Sequential()
model.add(Dense(units = 512, activation='relu', input_shape=(784,)))
model.add(Dense(units = 512, activation='relu'))
model.add(Dense(units = num_classes, activation='softmax'))
```

## 總結模型

執行下方儲存格來總結你剛剛建立的模型：

In [ ]:
model.summary()

## 編寫模型

我們要用和先前一樣的選項來[編寫](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#compile)模型，使用[分類交叉熵](https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy)以反映出我們的目的是讓輸出接近眾多類別的其中之一，並衡量模型的準確度：

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

## 練習：訓練模型

使用模型的 `fit`方法來對模型進行 20 個 Epoch 的訓練，使用的是先前建立的訓練與驗證影像和類別：

In [ ]:
# TODO: Train the model for 20 epochs.

### 解決方案

按一下以下的「...」以顯示解決方案。

```python
model.fit(x_train, y_train, epochs=20, verbose=1, validation_data=(x_valid, y_valid))
```

## 討論：發生什麼事了？

我們可以看到訓練準確度相當高，但驗證的準確度卻沒那麼高。怎麼回事？

請先思考一下，再按一下「...」以展開答案。

這個範例顯示出，模型學會將訓練資料分類，對於尚未在其訓練中出現的新資料卻表現不佳。本質上來說，模型只是記憶資料集，無法對問題有可靠且概略的理解。這是很常見的問題，也就是所謂的*過度擬合*。我們將在接下來的兩堂課程中討論過度擬合，以及處理這個問題的一些方法。

## 摘要

在本節中，你建立了自己的神經網路，並用於執行相當準確的影像分類。恭喜！

到目前為止，你應該已經漸漸熟悉載入資料 (包括影像類別)、準備資料、建立模型，接著使用準備好的資料訓練模型等流程。

### 清除記憶體
在繼續之前，請執行下列儲存格以清除 GPU 記憶體。必須先完成此步驟才能繼續進行下一個 Notebook。

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## 下一步

現在，你已經建立了一些非常基本且頗有效率的模型，接著我們要開始探討更複雜的模型，包括*卷積神經網路*。

請繼續前往下一節：[*使用卷積神經網路處理美國手語影像*](./03_asl_cnn.ipynb)。